# 실습 [16-1]<br>
**실습명: 스팸 햄 메일 분류**<br>

In [ ]:
!pip install np

  Created wheel for np: filename=np-1.0.2-cp37-none-any.whl size=13677 sha256=80dcdc98a1350766380b565222b291241b130c4e2ee24cd564e071f6c543ebf2
  Stored in directory: /root/.cache/pip/wheels/2b/df/57/f40bef951382112d9c644b3ec6d713ceb200ea7c15c074de42
Successfully built np


In [ ]:
#관련 라이브러리 불러오기
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import csv
import np #pip install np

#환경설정
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving SMSSpamCollection to SMSSpamCollection


In [ ]:
smsdata = open('SMSSpamCollection',encoding='utf8') #SMSSpamCollection의 경로 지정

In [ ]:
#1. 데이터 전처리 과정
def preprocessing(text):
    #단어 토큰화
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)] 
   
    #불용어 제거
    stop = stopwords.words('english')
    tokens = [token for token in tokens if token not in stop]
    
    #3글자보다 적은 글자를 가진 단어 제거 
    tokens = [word for word in tokens if len(word) >= 3]
    
    #모두 소문자로 변환
    tokens = [word.lower() for word in tokens]
    
    #어간 추출(lemmatization)
    lmtzr = WordNetLemmatizer()
    tokens = [lmtzr.lemmatize(word) for word in tokens]

    preprocessed_text= ' '.join(tokens)
    return preprocessed_text

In [ ]:
sms_data = []
sms_labels = []
cnt = 0
sencsv_reader = csv.reader(smsdata,delimiter='\t')
for line in sencsv_reader:
    sms_labels.append(line[0])
    sms_data.append(preprocessing(line[1]))

smsdata.close()

In [ ]:
#train-test 데이터로 분리
trainset_size = int(round(len(sms_data)*0.70)) #7:3으로 훈련:시험 데이터 분할
print('The training set size for this classifier is ' + str(trainset_size) + '\n')

x_train = np.array([''.join(el) for el in sms_data[0:trainset_size]]) #0~70%까지의 데이터 사용
y_train = np.array([el for el in sms_labels[0:trainset_size]])

x_test = np.array([''.join(el) for el in sms_data[trainset_size+1:len(sms_data)]]) #나머지 30% 데이터 사용
y_test = np.array(([el for el in sms_labels[trainset_size+1:len(sms_labels)]]) or el in sms_labels[trainset_size+1:len(sms_labels)])

The training set size for this classifier is 3900



In [ ]:
#특징값 추출: TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer2 = TfidfVectorizer(min_df=2, ngram_range=(1, 2), stop_words='english', strip_accents='unicode', norm='l2')
X_train = vectorizer2.fit_transform(x_train)
X_test = vectorizer2.transform(x_test)

In [ ]:
#텍스트 분류 학습 (Naive Bayes 분류기)
#다중분류 NB 분류기 사용 -> 문서 분류하는 경우에 유용함
from sklearn.naive_bayes import MultinomialNB
clf_NB = MultinomialNB().fit(X_train, y_train)
y_predicted_NB = clf_NB.predict(X_test)

In [ ]:
#Decision Tree 분류기
from sklearn import tree
clf_DT = tree.DecisionTreeClassifier().fit(X_train, y_train)
y_predicted_DT = clf_DT.predict(X_test.toarray())

In [ ]:
#Stochastic Gradient Descent (경사하강법)
from sklearn.linear_model import SGDClassifier
clf_SGD = SGDClassifier(alpha=0.001).fit(X_train, y_train)
y_predicted_SGD = clf_SGD.predict(X_test)

In [ ]:
#Support Vector Machines (SVM)
from sklearn.svm import LinearSVC
clf_SVM = LinearSVC().fit(X_train, y_train)
y_predicted_SVM = clf_SVM.predict(X_test)

In [ ]:
#The Random forest algorithm (앙상블 학습방법)
from sklearn.ensemble import RandomForestClassifier
clf_RFA = RandomForestClassifier(n_estimators=10)
clf_RFA.fit(X_train, y_train)
y_predicted_RFA = clf_RFA.predict(X_test)

In [ ]:
#각 모델간의 성능 검증
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

#Naive Bayes
print(' \n confusion_matrix NB \n ')
cm = confusion_matrix(y_test, y_predicted_NB)
print(cm)
print(' \n classification report NB \n ')
print(classification_report(y_test, y_predicted_NB))

#Decision Tree
print(' \n confusion_matrix DT \n ')
cm = confusion_matrix(y_test, y_predicted_DT)
print(cm)
print(' \n classification report DT \n ')
print(classification_report(y_test, y_predicted_DT))

#SGD
print(' \n confusion_matrix SGD \n ')
cm = confusion_matrix(y_test, y_predicted_SGD)
print(cm)
print(' \n classification report SGD \n ')
print(classification_report(y_test, y_predicted_SGD))

#SVM
print(' \n confusion_matrix SVM \n ')
cm = confusion_matrix(y_test, y_predicted_SVM)
print(cm)
print(' \n classification report SVM \n ')
print(classification_report(y_test, y_predicted_SVM))

#The Random Forest Algorithm
print(' \n confusion_matrix RFA \n ')
cm = confusion_matrix(y_test, y_predicted_RFA)
print(cm)
print(' \n classification report RFA \n ')
print(classification_report(y_test, y_predicted_RFA))

 
 confusion_matrix NB 
 
[[1443    0]
 [  52  176]]
 
 classification report NB 
 
              precision    recall  f1-score   support

         ham       0.97      1.00      0.98      1443
        spam       1.00      0.77      0.87       228

    accuracy                           0.97      1671
   macro avg       0.98      0.89      0.93      1671
weighted avg       0.97      0.97      0.97      1671

 
 confusion_matrix DT 
 
[[1417   26]
 [  46  182]]
 
 classification report DT 
 
              precision    recall  f1-score   support

         ham       0.97      0.98      0.98      1443
        spam       0.88      0.80      0.83       228

    accuracy                           0.96      1671
   macro avg       0.92      0.89      0.91      1671
weighted avg       0.96      0.96      0.96      1671

 
 confusion_matrix SGD 
 
[[1437    6]
 [  41  187]]
 
 classification report SGD 
 
              precision    recall  f1-score   support

         ham       0.97      1.00    